In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import shfl
import torch
import copy
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0-Segmentadas", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 50,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

In [3]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0]), 
                'P' : lb1.transform(['P'])[0][0], 'N' : lb1.transform(['N'])[0][0]
              } 



In [4]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", folds=1, lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder( G_dict ):
    return ClassifierModel(G_dict, dict_labels, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels


    return t_federated_data

In [5]:
print("[INFO] Fetching federated data...")
federated_data, train_data, train_label, test_data, test_label, train_files, test_files, args["num_nodes"] = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)
federated_data.configure_data_access(UnprotectedAccess())
print(len(train_data))
print(len(test_data))
print("[INFO] done")

[INFO] Fetching federated data...
681
171
[INFO] done


In [6]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
cit_federated_government = shfl.federated_government.FederatedGovernment(cit_builder, federated_data, aggregator)
cit_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

Accuracy round 0


  0%|          | 0/23 [00:00<?, ?it/s]

Training node 0
[INFO] weights = [1.         0.93965517]


[Validating]: Acc_D: 0.2609: 100%|██████████| 23/23 [00:00<00:00, 28.40it/s]


[INFO] LAMBDA: 0.05


...


Valid Acc = 0.875
Valid Loss = 0.4670220084177951
EarlyStopping counter: 9 out of 10


[29/50] Loss_D: 0.1323 Acc_D: 0.8905 Loss_G_class1: 0.0086 Loss_G_class2: 0.0083: 100%|██████████| 27/27 [00:40<00:00,  1.51s/it]
  0%|          | 0/24 [00:00<?, ?it/s]


Valid Acc = 0.8333333333333334
Valid Loss = 0.6151562499192854
EarlyStopping counter: 10 out of 10
Early stopping, epoch 29


[Validating]: Acc_D: 0.8750: 100%|██████████| 24/24 [00:00<00:00, 31.69it/s]


[INFO] Summary of training for LAMBDA = 0.05 (best model values)
Valid Acc = 0.875
Valid Loss = 0.505636791077753
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f03425a99e8>: [0.7426223094351807, 0.6608187134502924, {'0': {'precision': 0.9444444444444444, 'recall': 0.6159420289855072, 'f1-score': 0.7456140350877193, 'support': 138}, '1': {'precision': 0.345679012345679, 'recall': 0.8484848484848485, 'f1-score': 0.4912280701754386, 'support': 33}, 'accuracy': 0.6608187134502924, 'macro avg': {'precision': 0.6450617283950617, 'recall': 0.7322134387351779, 'f1-score': 0.618421052631579, 'support': 171}, 'weighted avg': {'precision': 0.8288932207060863, 'recall': 0.6608187134502924, 'f1-score': 0.6965220067713143, 'support': 171}}]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f03425a9710>: [1.1105456775280962, 0.7543859649122807, {'0': {'precision': 0.9666666666666667, 'recall': 0.6904761904761905, 'f1-sc

In [11]:
metrics = cit_federated_government.global_model.evaluate(test_data, test_label)
print("CIT Classifier Results:")
print("Loss: {}".format(metrics[0]))
print("Acc: {}".format(metrics[1]))
print(metrics[2])

CIT Classifier Results:
Loss: 0.7426223080136884
Acc: 0.6608187134502924
{'0': {'precision': 0.9444444444444444, 'recall': 0.6159420289855072, 'f1-score': 0.7456140350877193, 'support': 138}, '1': {'precision': 0.345679012345679, 'recall': 0.8484848484848485, 'f1-score': 0.4912280701754386, 'support': 33}, 'accuracy': 0.6608187134502924, 'macro avg': {'precision': 0.6450617283950617, 'recall': 0.7322134387351779, 'f1-score': 0.618421052631579, 'support': 171}, 'weighted avg': {'precision': 0.8288932207060863, 'recall': 0.6608187134502924, 'f1-score': 0.6965220067713143, 'support': 171}}


In [12]:
t_federated_data = get_transformed_data(federated_data, cit_federated_government, lb1, lb2)

In [13]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
G_dict = cit_federated_government.global_model._G_dict
x = lambda : classifier_builder(G_dict)
classifier_federated_government = shfl.federated_government.FederatedGovernment(x, t_federated_data, aggregator)
#classifier_federated_government = FederatedGovernment_2(classifier_builder(G_dict), t_federated_data, aggregator)
classifier_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

#print("[INFO] saving model ...")
#federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
print("[INFO] done")

Accuracy round 0
Training node 0
Epoch 1/50
50/50 [==============================] - 24s 489ms/step - loss: 0.9277 - categorical_accuracy: 0.5340 - val_loss: 0.9294 - val_categorical_accuracy: 0.5250
Epoch 2/50
50/50 [==============================] - 4s 73ms/step - loss: 0.4947 - categorical_accuracy: 0.7859 - val_loss: 1.4414 - val_categorical_accuracy: 0.5500
Epoch 3/50
50/50 [==============================] - 4s 73ms/step - loss: 0.2868 - categorical_accuracy: 0.9093 - val_loss: 1.0637 - val_categorical_accuracy: 0.5750
Epoch 4/50
50/50 [==============================] - 4s 71ms/step - loss: 0.1358 - categorical_accuracy: 0.9622 - val_loss: 1.5693 - val_categorical_accuracy: 0.5500
Epoch 5/50
50/50 [==============================] - 4s 72ms/step - loss: 0.0879 - categorical_accuracy: 0.9798 - val_loss: 1.6455 - val_categorical_accuracy: 0.5500
Epoch 6/50
50/50 [==============================] - 4s 74ms/step - loss: 0.0811 - categorical_accuracy: 0.9723 - val_loss: 0.8976 - val_cate

Epoch 00049: early stopping
Training node 1
Epoch 1/50
49/49 [==============================] - 27s 547ms/step - loss: 0.8625 - categorical_accuracy: 0.5995 - val_loss: 1.2031 - val_categorical_accuracy: 0.4500
Epoch 2/50
49/49 [==============================] - 4s 73ms/step - loss: 0.4763 - categorical_accuracy: 0.7985 - val_loss: 0.7243 - val_categorical_accuracy: 0.6750
Epoch 3/50
49/49 [==============================] - 4s 82ms/step - loss: 0.2043 - categorical_accuracy: 0.9413 - val_loss: 1.0352 - val_categorical_accuracy: 0.6500
Epoch 4/50
49/49 [==============================] - 4s 73ms/step - loss: 0.1585 - categorical_accuracy: 0.9515 - val_loss: 0.6011 - val_categorical_accuracy: 0.7750
Epoch 5/50
49/49 [==============================] - 4s 73ms/step - loss: 0.0621 - categorical_accuracy: 0.9898 - val_loss: 0.7656 - val_categorical_accuracy: 0.8000
Epoch 6/50
49/49 [==============================] - 4s 73ms/step - loss: 0.0348 - categorical_accuracy: 0.9974 - val_loss: 0.7060

Epoch 23/50
52/52 [==============================] - 4s 75ms/step - loss: 0.0096 - categorical_accuracy: 1.0000 - val_loss: 0.1914 - val_categorical_accuracy: 0.9500
Epoch 24/50
52/52 [==============================] - 4s 77ms/step - loss: 0.0111 - categorical_accuracy: 0.9952 - val_loss: 0.1867 - val_categorical_accuracy: 0.8750
Epoch 25/50
52/52 [==============================] - 4s 76ms/step - loss: 0.0066 - categorical_accuracy: 1.0000 - val_loss: 0.1080 - val_categorical_accuracy: 0.9750
Epoch 26/50
52/52 [==============================] - 4s 75ms/step - loss: 0.0103 - categorical_accuracy: 0.9976 - val_loss: 0.4009 - val_categorical_accuracy: 0.8750
Epoch 27/50
52/52 [==============================] - 4s 75ms/step - loss: 0.0275 - categorical_accuracy: 0.9879 - val_loss: 0.3844 - val_categorical_accuracy: 0.8500
Epoch 28/50
52/52 [==============================] - 4s 81ms/step - loss: 0.0195 - categorical_accuracy: 0.9976 - val_loss: 0.3591 - val_categorical_accuracy: 0.8750
Epoc

In [10]:
metrics = classifier_federated_government.global_model.evaluate(test_data, test_label)
print("SDNET Classifier Results:")
print("Acc: {}".format(metrics[0]))
print("Acc_4: {}".format(metrics[1]))
print("No concuerda: {}".format(metrics[2]))
print(metrics[3])

SDNET Classifier Results:
Acc: 0.7543859649122807
Acc_4: 0.7660818713450293
No concuerda: 20
{'0': {'precision': 0.7264150943396226, 'recall': 0.8555555555555555, 'f1-score': 0.7857142857142856, 'support': 90}, '1': {'precision': 0.8, 'recall': 0.6419753086419753, 'f1-score': 0.7123287671232877, 'support': 81}, 'accuracy': 0.7543859649122807, 'macro avg': {'precision': 0.7632075471698113, 'recall': 0.7487654320987653, 'f1-score': 0.7490215264187867, 'support': 171}, 'weighted avg': {'precision': 0.761271102284012, 'recall': 0.7543859649122807, 'f1-score': 0.7509527242764447, 'support': 171}}
